In [1]:
import os
import mysql.connector
import pandas as pd
from datetime import datetime
import dateutil.relativedelta as dr

Conexão com MySQL

In [2]:
conexao = mysql.connector.connect(host=os.getenv('DW_HOST'), user=os.getenv('DW_USER'), password=os.getenv('DW_PASSWORD'), database=os.getenv('DW_DB'))

In [3]:
conexao

Leitura das queries que serão utilizadas

In [4]:
with open('./queries/total_tickets.sql', 'r', encoding='utf-8') as line:
    total_tickets = line.read()

In [5]:
with open('./queries/tickets_rota.sql', 'r', encoding='utf-8') as line:
    tickets_rota = line.read()

In [6]:
with open('./queries/destinos_spturins.sql', 'r', encoding='utf-8') as line:
    top5 = line.read()

Ler e salvar as queries dentro de variáveis passando o nome da query e o nome da variável que faz a conexão com o banco

In [ ]:
tickets_totais = pd.read_sql_query(total_tickets, conexao)

In [ ]:
tickets_totais

In [ ]:
tickets_rotas = pd.read_sql_query(tickets_rota, conexao)

In [ ]:
tickets_rotas

Leitura do CSV para fazer a ordenação do resultado final

In [ ]:
ordem = pd.read_csv('./csv/order.csv')
ordem['order'] = ordem.index+1

Junção da query de quantidade de tickets por rota com a "ordem" e ordenação pela coluna de index

In [ ]:
resultado = pd.merge(tickets_rotas, ordem, how='left', left_on='sk_route', right_on='id')
resultado = resultado.sort_values(by='order')

In [ ]:
resultado

Inserindo coluna com o volume por rota

In [ ]:
resultado['Volume'] = round((resultado['Tickets rota']/tickets_totais.values[0][0])*100, 2)

Exclusão de colunas que não precisam ser mostradas no dataframe

In [ ]:
resultado = resultado.drop(columns=['sk_route', 'Tickets rota', 'id', 'order'])

In [ ]:
resultado.head()

Dataframe para um arquivo csv na pasta "csv"

In [ ]:
resultado.to_csv('/home/nicole/DataspellProjects/dsProject/PR/csv/volume_spturins.csv', index=False)

Leitura da query de top 5 destinos do mês anterior

In [ ]:
top_destinos = pd.read_sql_query(top5,conexao)

In [ ]:
top_destinos

Mostrar o mês anterior e o ano atual

In [ ]:
now = datetime.now() #data atual
x = now + dr.relativedelta(months=-1) #mês anterior
y = (x.strftime("%b") + '/' + x.strftime("%y") ) # %b para mostrar o mês e %y para o ano

Alteração do nome da coluna

In [ ]:
top_destinos.rename(columns={'Top 5 destinos mais buscados': 'Top 5 destinos mais buscados em {}'.format(y)}, inplace = True)

Exclusão de colunas que não precisam ser mostradas no dataframe

In [ ]:
top_destinos.drop(columns={'COUNT(b.ticket_success)'}, inplace=True)

Dataframe para um arquivo csv na pasta "csv"

In [ ]:
top_destinos.to_csv('/home/nicole/DataspellProjects/dsProject/PR/csv/topdestinos_spturins.csv', index=False)